In [61]:
# %load_ext autoreload
# %autoreload 2
import models, utils
import orca
import dask.dataframe as dd
from __future__ import division
import csv
import numpy as np
import pandas as pd
import logging
import greenroof

In [62]:
import numpy as np
from pandas import HDFStore,DataFrame
import h5py
filename = 'C:\GreenRoof\harrisonfreundgreenroofrepository\sanfran_urbansim-master\data\sanfran_public.h5'
with h5py.File(filename, 'r') as hdf:
    ls = list(hdf.keys())
    print('Datasets: \n', ls)
    data = hdf.get('buildings')
    dataset1 = np.array(data)


Datasets: 
 ['buildings', 'households', 'jobs', 'parcels', 'zones', 'zoning', 'zoning_for_parcels']


In [63]:
#!/usr/bin/env python
'''Reads NeXus HDF5 files using h5py and prints the contents'''

import h5py    # HDF5 support

fileName = 'C:\GreenRoof\harrisonfreundgreenroofrepository\sanfran_urbansim-master\data\sanfran_public.h5'
f = h5py.File(fileName,  "r")
print(f)
f
#for item in f.attrs.keys():
#    print(item + ":", f.attrs[item])
#mr = f['/entry/mr_scan/mr']
#i00 = f['/entry/mr_scan/I00']
#print("%s\t%s\t%s" % ("#", "mr", "I00"))
#for i in range(len(mr)):
 #   print("%d\t%g\t%d" % (i, mr[i], i00[i]))
#f.close()

<HDF5 file "sanfran_public.h5" (mode r)>


<HDF5 file "sanfran_public.h5" (mode r)>

In [4]:
buildings = orca.get_table('buildings')
buildings.to_frame()
print(orca.list_columns())
mybuild = orca.orca.DataFrameWrapper.get_column(buildings, 'unit_sqft')
mybuild.to_frame()
mybuild.sort_index(inplace=True)
mybuild.to_frame()
height = orca.orca.DataFrameWrapper.get_column(buildings, 'stories')
height.sort_index(inplace=True)
height.to_frame()
dataset = pd.concat([height, mybuild], axis=1)
#height of building approx = 11.5 feet * stories

Filling column residential_units with value 0 (0 values)
Filling column non_residential_sqft with value 0 (1341 values)
Filling column building_type_id with value 2.0 (0 values)
Filling column non_residential_rent with value 0 (142400 values)
Filling column residential_sales_price with value 0 (14196 values)
Filling column year_built with value 1927.0 (3116 values)
Filling column job_category with value service (331 values)
[('zones_prices', 'zone_id'), ('parcels', 'parcel_size'), ('zones', 'hhsize'), ('buildings', 'vacant_residential_units'), ('parcels', 'ave_unit_size'), ('parcels', 'land_cost'), ('buildings', 'sqft_per_job'), ('parcels', 'total_units'), ('zones', 'zone_id'), ('zones', 'jobs'), ('buildings', 'unit_sqft'), ('buildings', 'job_spaces'), ('zones', 'sfdu'), ('buildings', 'zone_id'), ('zones', 'ave_income'), ('zones', 'poor'), ('households', 'zone_id'), ('zones', 'sum_residential_units'), ('zones', 'renters'), ('households', 'income_quartile'), ('zones_prices', 'industrial

In [3]:
buildings = orca.get_table('jobs')
buildings.to_frame()

Filling column job_category with value service (331 values)
Filling column residential_units with value 0 (0 values)
Filling column non_residential_rent with value 0 (142400 values)
Filling column year_built with value 1927.0 (3116 values)
Filling column non_residential_sqft with value 0 (1341 values)
Filling column residential_sales_price with value 0 (14196 values)
Filling column building_type_id with value 2.0 (0 values)


,building_id,job_category,zone_id
3389004,1845867,service,107.0
3813527,1854491,service,14.0
3530098,1849820,service,65.0
3417867,1854265,service,4.0
3585435,1854265,service,4.0
3816089,1854591,service,12.0
3380206,1854058,service,2.0
3458689,1853110,service,32.0
3991324,131004,service,11.0
3619366,1852886,service,47.0


In [112]:
#TO DO: Gradually Transfer this alg to data cell below

data = pd.read_csv('data.csv')
print(data['sq_ft'][0])
roofinstalled_dict = {}
roof_cost_at_20yrs_dict = {}
green_roof_benefit_dict = {}
green_roof_cost_dict = {}
green_roof_net_benefit_dict = {}     
building_size_dict = {}
#greenroof.NoInterventionGreenRoof()


for index in range(-1, 1000, 1):
    index = index + 1
    building_size = (data['sq_ft'][index])/(data['stories'][index])
    building_size_dict.update({index : building_size})
    #Initialization of assumed and derived variables
    base_green_cost_per_sqft = 12
    green_roof_min = 0
    conventional_cost_per_sqft = green_cost_per_sqft/2
    green_roof_benefits = 0
    building_age = 40
    annual_benefit_per_area = 1.23
    annual_maintenance_costs_per_area = 0.02 
    min_green_area = green_roof_min * building_size
    min_green_cost = green_roof_min * green_cost_per_sqft
    best_green_cost = min_green_cost #default value
    min_green_benefits= green_roof_min*building_size*annual_benefit_per_area*building_age
    best_green_net_benefit = min_green_benefits #default value
    best_size = green_roof_min
    best_benefit = min_green_benefits - min_green_cost
    best_conv = (building_size*(1-green_roof_min)*conventional_cost_per_sqft) #Avoid capital costs
    roofinstalled_dict.update({index : 1}) #this green roof variable can be stored in the final



    #Calculation of Output variables

    #If the building is large enough to allow bulk discounting to begin (happens in most cases)
    if (building_size > 500):
        green_cost_per_sqft = 23.907*(base_green_cost_per_sqft**(-0.078))

    #for current_size in my_range(green_roof_min, 1, 0.1):
    #    roof_cost_at_20yrs = (building_size*(1-current_size)*conventional_cost_per_sqft) #Avoid capital costs
    #    green_roof_cost = current_size * building_size * green_cost_per_sqft + (building_age * annual_maintenance_costs_per_area * building_size * current_size)
    #    green_roof_benefits = building_age * annual_benefit_per_area * building_size * current_size + roof_cost_at_20yrs
    #    if(green_roof_benefits - green_roof_cost > best_benefit):
    #        best_size = current_size
    #        best_green_cost = green_roof_cost
    #        best_green_benefits = green_roof_benefits
    #        best__green_net_benefit = green_roof_benefits - green_roof_cost
    #        best_conv = roof_cost_at_20yrs

    if(best_benefit > 0):
        roofinstalled_dict.update({index : 1}) #this green roof variable can be stored in the final
    else:
        roofinstalled_dict.update({index : 0})

    roof_cost_at_20yrs_dict.update({index : best_conv})
    green_roof_benefit_dict.update({index : best_benefit})
    green_roof_cost_dict.update({index : best_green_cost})
    green_roof_net_benefit_dict.update({index : best_green_net_benefit})
    index = index +1    

nan


C:\Users\Harrison Freund\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#WORK SHOULD BE DONE IN HERE

def my_range(start, end, step):
    while start <= end:
        yield start #keyword yield generates an interator
        start += step


building_size_dict
roofins_dict = {}
green_roof_cost_dict = {}
roof_cost_at_20yrs_dict = {}
size_dict = {}
for index in range(-1, 100000, 1):
    index = index + 1
    building_size = (data['sq_ft'][index])/(data['stories'][index])
    building_size_dict.update({index : building_size})
    roof_ins = 0
    best_size = 0
    for size_ratio in my_range (0, 1, .01):
        green_roof_cost = 23.907*(12**(-0.078)) *size_ratio * building_size
        green_roof_ben = 1.23 * size_ratio * building_size
        if(green_roof_cost < green_roof_ben):
            roofins = 1
            best_size = size_ratio * building_size
    size_dict.update({index : best_size})
    roofins_dict.update({index : roofins})
    green_roof_cost_dict.update({index : green_roof_cost})
    roof_cost_at_20yrs = building_size * 6
    roof_cost_at_20yrs_dict.update({index : roof_cost_at_20yrs})
    
buildings = pd.DataFrame({"building size" : building_size_dict, "best_size" : size_dict, "roofins" : roofins_dict, "green_cost" : green_roof_cost_dict, "cost_20yr" : roof_cost_at_20yrs_dict})
writer = pd.ExcelWriter('no_action.xlsx')
buildings.to_excel(writer,'no_action')
writer.save()

In [134]:
def my_range(start, end, step):
    while start <= end:
        yield start
        start += step

for x in my_range(1, 10, 0.5):
    print (x)

1
1.5
2.0
2.5
3.0
3.5
4.0
4.5
5.0
5.5
6.0
6.5
7.0
7.5
8.0
8.5
9.0
9.5
10.0


In [ ]:
parcels = orca.get_table('parcels')
parcels.to_frame()

In [ ]:
buildings = orca.get_table('buildings')
buildings.to_frame()

In [ ]:
zones = orca.get_table('zones')
zones.to_frame()

In [57]:
orca.run([
    "rsh_simulate",              # residential sales hedonic
    "nrh_simulate",              # non-residential rent hedonic

    "households_relocation",     # households relocation model
    "hlcm_simulate",            # households location choice
    "households_transition",     # households transition

    "jobs_relocation",           # jobs relocation model
    "elcm_simulate",             # employment location choice
    "jobs_transition",           # jobs transition

    "feasibility",               # compute development feasibility
    "residential_developer",     # build residential buildings
    "non_residential_developer", # build non-residential buildings
], iter_vars=range(2010-2015) #[2010, 2011, 2012, 2013, 2014])

Running iteration 1 with iteration value 2010
Running step 'rsh_simulate'
count    1.450110e+05
mean     1.125600e+03
std      3.339318e+04
min      2.137348e-03
25%      2.652641e+02
50%      4.015027e+02
75%      6.029871e+02
max      9.785525e+06
dtype: float64
Time to execute step 'rsh_simulate': 0.88 s
Running step 'nrh_simulate'
count    10803.000000
mean        20.683461
std          6.822014
min          5.891729
25%         17.305616
50%         20.869273
75%         24.901835
max         46.964186
dtype: float64
Time to execute step 'nrh_simulate': 0.55 s
Running step 'households_relocation'
Total agents: 400060
Total currently unplaced: 19050
Assinging for relocation...
Total currently unplaced: 38142
Time to execute step 'households_relocation': 0.08 s
Running step 'hlcm_simulate'
There are 463236.0 total available units
    and 400060 total choosers
    but there are 32587 overfull buildings
    for a total of 147407 temporarily empty units
    in 57670 buildings total in 

C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\developer\sqftproforma.py:394: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ave_cost_sqft'][c.fars > c.max_industrial_height] = np.nan
C:\Users\Harrison Freund\Anaconda3\lib\site-packages\urbansim\developer\sqftproforma.py:392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['ave_cost_sqft'][c.fars > c.max_retail_height] = np.nan


Describe of the yearly rent by use
              retail    industrial         office    residential
count  152205.000000  94493.000000  152259.000000  153228.000000
mean       23.924756      8.567153      21.322894      20.724450
std         3.538226      1.652726       2.999300       6.276502
min        18.000000      5.000000      16.000000       1.175000
25%              NaN           NaN            NaN            NaN
50%              NaN           NaN            NaN            NaN
75%              NaN           NaN            NaN            NaN
max        35.000000     14.500000      30.000000      70.975000
Computing feasibility for form mixedoffice
Computing feasibility for form mixedresidential


C:\Users\Harrison Freund\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


Computing feasibility for form residential
Computing feasibility for form industrial
Computing feasibility for form retail
Computing feasibility for form office
Time to execute step 'feasibility': 2.96 s
Running step 'residential_developer'
Number of agents: 420,063
Number of agent spaces: 463,236
Current vacancy = 0.09
Target vacancy = 0.15, target of new units = 30,955
2,117 feasible buildings before running developer
Sum of net units that are profitable: 7,931
WARNING THERE WERE NOT ENOUGH PROFITABLE UNITS TO MATCH DEMAND
Adding 670 buildings with 8,824 residential_units
1,447 feasible buildings after running developer
Time to execute step 'residential_developer': 0.34 s
Running step 'non_residential_developer'
Number of agents: 274,023
Number of agent spaces: 642,004
Current vacancy = 0.57
Target vacancy = 0.15, target of new units = 0
1,447 feasible buildings before running developer
Sum of net units that are profitable: 81,924
Adding 0 buildings with 0 job_spaces
1,447 feasible b